In [1]:
from config.Experiment_Config import ExperimentConfig
from exp.trainer import Trainer

c:\Users\yunkai\.conda\envs\Multi_turn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = ExperimentConfig.get_default_config()
config.training_settings.num_epochs = 1000
config.training_settings.batch_size = 32
config.tokenizer_settings.max_length = 256
config.model_settings.weight_init = 'kaiming_normal'
config.model_settings.init_hidden_dim = 384
config.model_settings.activation = 'gelu'
config.model_settings.fine_tune_embedding = False
config.training_settings.early_stopping_patience = 100
config.training_settings.gradient_clip = None
config.training_settings.task_type = 'classification'
config.model_settings.num_layers = 4
config.model_settings.custom_hidden_dims = [768, 384, 192, 96]
config.model_settings.use_res_net = True
config.model_settings.use_attention = False
config.training_settings.gradient_clip = 1.0
config.training_settings.continue_training = True
config.training_settings.learning_rate = 5e-5
config.model_settings.use_layer_norm = True
config.model_settings.fine_tune_embedding = True


In [3]:
trainer = Trainer(
    config=config
)

# Train model
metrics_history = trainer.train()

d:\Program\Py\project\Multi_turn\models\LSTM\BaseLSTM.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)



[Fine-tuning] Loaded embedding model from epoch 10
✅ Created all necessary directories

EXPERIMENT CONFIGURATION

📊 Experiment Name: CLS_ResBiLSTM_BERT_L4_H384_M256_LR5e-5_Actgelu_crossentropy_kaiming_FTLN
📁 Model Directory: saved_models\CLS_ResBiLSTM_BERT_L4_H384_M256_LR5e-5_Actgelu_crossentropy_kaiming_FTLN
📈 Results Directory: results\CLS_ResBiLSTM_BERT_L4_H384_M256_LR5e-5_Actgelu_crossentropy_kaiming_FTLN

🔧 Model Configuration:
- Model Type: BiLSTM
- Embedding Type: BERT_base_uncased
- Hidden Dimensions: [768, 384, 192, 96]
- Bidirectional: True
- Dropout Rate: 0.1
- Attention: False
- Fine-tune Embedding: True
- Weight Init: kaiming_normal
- Activation: gelu

⚙️ Training Configuration:
- Batch Size: 32
- Max Length: 256
- Learning Rate: 5e-05
- Weight Decay: 0.01
- Num Epochs: 1000
- Loss Function: cross_entropy
- Optimizer: adamw
- Gradient Clipping: 1.0
- Early Stopping Patience: 100
- Scheduler Patience: 3
- Scheduler Factor: 0.1
- Min Learning Rate: 1e-06
- Checkpoint Freque

Evaluating (val): 100%|██████████| 8/8 [00:01<00:00,  6.71it/s]



📊 Train Metrics:
Loss: 3.6123 | Accuracy: 0.0253
Macro → P: 0.0061 | R: 0.0221 | F1: 0.0089
Micro → P: 0.0253 | R: 0.0253 | F1: 0.0253



📊 Validation Metrics:
Loss: 3.6067 | Accuracy: 0.0430
Macro → P: 0.0012 | R: 0.0270 | F1: 0.0022
Micro → P: 0.0430 | R: 0.0430 | F1: 0.0430


📈 Saved latest metrics plot at epoch 1
🔄 Saved latest confusion matrices at epoch 1
📊 Saved latest performance plots at epoch 1
📈 Saved best metrics plot at epoch 1
🔄 Saved best confusion matrices at epoch 1
📊 Saved best performance plots at epoch 1
📝 Saved latest model at epoch 1
🏆 Saved best model at epoch 1

──────────────────────────────────────────────────
⏳ Epoch 2/1000
──────────────────────────────────────────────────


Evaluating (val): 100%|██████████| 8/8 [00:03<00:00,  2.22it/s]


...


📊 Train Metrics:
Loss: 3.6045 | Accuracy: 0.0377
Macro → P: 0.0038 | R: 0.0238 | F1: 0.0057
Micro → P: 0.0377 | R: 0.0377 | F1: 0.0377



📊 Validation Metrics:
Loss: 3.5997 | Accuracy: 0.0469
Macro → P: 0.0013 | R: 0.0270 | F1: 0.0024
Micro → P: 0.0469 | R: 0.0469 | F1: 0.0469


📈 Saved latest metrics plot at epoch 26
🔄 Saved latest confusion matrices at epoch 26
📊 Saved latest performance plots at epoch 26
📝 Saved latest model at epoch 26

──────────────────────────────────────────────────
⏳ Epoch 27/1000
──────────────────────────────────────────────────


Evaluating (val): 100%|██████████| 8/8 [00:03<00:00,  2.15it/s]



📊 Train Metrics:
Loss: 3.6030 | Accuracy: 0.0427
Macro → P: 0.0047 | R: 0.0273 | F1: 0.0077
Micro → P: 0.0427 | R: 0.0427 | F1: 0.0427



📊 Validation Metrics:
Loss: 3.5995 | Accuracy: 0.0469
Macro → P: 0.0013 | R: 0.0270 | F1: 0.0024
Micro → P: 0.0469 | R: 0.0469 | F1: 0.0469


📈 Saved latest metrics plot at epoch 27
🔄 Saved latest confusion matrices at epoch 27
📊 Saved latest performance plots at epoch 27
📝 Saved latest model at epoch 27

──────────────────────────────────────────────────
⏳ Epoch 28/1000
──────────────────────────────────────────────────


Training:  81%|████████  | 51/63 [01:09<00:16,  1.36s/it]


KeyboardInterrupt: 